In [8]:
import os
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils import (
    load_image_and_masks,
    detect_background,
    clean_mask
)

DATA_ROOT = Path('./data')

IMAGES_ROOT = DATA_ROOT / 'images'
MASKS_DIR = DATA_ROOT / 'car_masks'
SHADOW_DIR = DATA_ROOT / 'shadow_masks'

def extract_car_image(image, mask):
    mask = clean_mask(mask)
    extracted_car = cv2.bitwise_and(image, image, mask=mask)
    return extracted_car

def create_background(floor, wall):
    start_row_floor, end_row_floor = detect_background(floor)
    _, end_row_wall = detect_background(wall)
    background = np.vstack((wall[int(end_row_wall-0.7*end_row_wall):end_row_wall, :], floor[start_row_floor:end_row_floor, :]))
    return background

def extract_car_images(car_image, car_mask):
    car_mask = clean_mask(car_mask)
    extracted_car = extract_car_image(car_image, car_mask)
    return extracted_car

def compose_image(image_path):
    car_image, car_mask, shadow_mask = load_image_and_masks(image_path)
    print(car_image.shape, car_mask.shape, shadow_mask.shape)
    floor = cv2.imread(DATA_ROOT / 'floor.png', cv2.IMREAD_UNCHANGED)
    print(floor.shape)
    wall = cv2.imread(DATA_ROOT / 'wall.png', cv2.IMREAD_UNCHANGED)
    print(wall.shape)
    background = create_background(floor, wall)
    cv2.imwrite('background.jpeg', background)
    extracted_car = extract_car_images(car_image, car_mask)
    cv2.imwrite('car.png', extracted_car)

if __name__ == '__main__':
    compose_image(IMAGES_ROOT / '3.jpeg')

(600, 800, 3) (600, 800) (170, 719)
(2160, 3840, 4)
(2160, 3840, 4)


In [10]:
for img in IMAGES_ROOT.glob('*.jpeg'):
    image, mask, shadow_mask = load_image_and_masks(img)
    print(f"{img} - {image.shape} - {mask.shape} - {shadow_mask.shape}")

data/images/4.jpeg - (1536, 2048, 3) - (1536, 2048) - (214, 1536)
data/images/1.jpeg - (1095, 1460, 3) - (1095, 1460) - (542, 1116)
data/images/3.jpeg - (600, 800, 3) - (600, 800) - (170, 719)
data/images/2.jpeg - (768, 1024, 3) - (768, 1024) - (217, 519)
data/images/6.jpeg - (1080, 1920, 3) - (1080, 1920) - (455, 1018)
data/images/5.jpeg - (768, 1024, 3) - (768, 1024) - (272, 679)


In [7]:
import os
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils import (
    load_image_and_masks,
    detect_background,
    clean_mask
)

DATA_ROOT = Path('./data')

IMAGES_ROOT = DATA_ROOT / 'images'
MASKS_DIR = DATA_ROOT / 'car_masks'
SHADOW_DIR = DATA_ROOT / 'shadow_masks'

def extract_car_image(image, mask):
    mask = clean_mask(mask)
    extracted_car = cv2.bitwise_and(image, image, mask=mask)
    return extracted_car

def create_background(floor, wall):
    start_row_floor, end_row_floor = detect_background(floor)
    start_row_wall, end_row_wall = detect_background(wall)
    background = np.vstack((wall[:end_row_wall, :], floor[start_row_floor:, :]))
    return background

def extract_car_images(car_image, car_mask):
    car_mask = clean_mask(car_mask)
    extracted_car = extract_car_image(car_image, car_mask)
    return extracted_car

def compose_image(image_path):
    car_image, car_mask, shadow_mask = load_image_and_masks(image_path)
    floor = cv2.imread(DATA_ROOT / 'floor.png', cv2.IMREAD_UNCHANGED)
    wall = cv2.imread(DATA_ROOT / 'wall.png', cv2.IMREAD_UNCHANGED)
    background = create_background(floor, wall)
    car_mask = clean_mask(car_mask)
    car_mask_reshaped = np.expand_dims(car_mask, axis=-1)
    print(car_image.shape, car_mask_reshaped.shape, background.shape)
    extracted_car = np.concatenate((car_image, car_mask_reshaped), axis=-1)
    st_row, end_row = detect_background(extracted_car)
    st_col, end_col = detect_background(extracted_car, axis=0)
    st_row -= 5
    end_row += 5
    st_col -= 5
    end_col += 5
    print(st_row, end_row, st_col, end_col)
    extracted_car = extracted_car[st_row:end_row, st_col:end_col, :]
    print(extracted_car.shape, shadow_mask.shape)
    # cv2.imwrite('car_extracted.png', extracted_car)
    # cv2.imwrite('background.png', background)
    print('done')

if __name__ == '__main__':
    compose_image(IMAGES_ROOT / '1.jpeg')

here
(1095, 1460, 3) (1095, 1460, 1) (2818, 3840, 4)
251 965 126 1335
(714, 1209, 4) (542, 1116)
done


In [9]:
cv2.imread(DATA_ROOT / 'sample-output.jpg', cv2.IMREAD_UNCHANGED).shape

(1080, 1920, 3)